# DogCatsRedux Kaggle Competition

### 1. Import required modules

In [1]:
# Rather than importing everything manually, we'll make things easy
# and load them all in utils.py, and just import them from there.

# 1. install bcolz (pip install bcolz)
# 2. install theano (pip install theano)
# 3. install keras (pip install keras)
# 4. install tensorflow (pip install tensorflow), (pip install tensorflow-gpu)
# 5. install protoclbuff (pip install protobuf)
%matplotlib inline 
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


### 2. Load the data (with augmentation)

In [2]:
base_path = '/home/ubuntu/data/redux/' #workspace directory
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [3]:
batch_size=100

In [4]:
batches = get_batches(base_path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(base_path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [6]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

#### Check the shape of the labels 

In [7]:
trn_labels.shape

(23000, 2)

In [8]:
val_labels.shape

(2000, 2)

### 3. Load the VGG model

In [9]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

### 5. No Dropout + Batch normalization + Data Augmentation

Get the convolution model and the fully connected model, we don't need to retrain conv model, so we get the output of the conv model and use this as input to dense, fully connected model. that way when we retrain the fc model, the input doesn't have to fall through all the conv layers

VGG predicts 1000 classes, but we need only 2 classes(cats/dogs). So pop the last Dense layer and add a new DenseLayer of size 2. Also retrain all dense layers.

In [10]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

Find the last conv layer

In [70]:
layers = vgg.model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

Split conv and fully connected layers

In [71]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

A function to make weights half, this is done since the VGG model had dropout of 50% and since we are going to remove it to avoid underfitting

In [72]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

Define the optimization algorithm

In [10]:
#opt = RMSprop(lr=0.1, rho=0.7) #what is rho?
opt = Adam()

Create a model for fully conected layers. the architecture is similar to original vgg fc part

In [74]:
def get_fc_bn_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        #BatchNormalization(mode=2),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        #BatchNormalization(mode=2),
        Dense(2, activation='softmax')
        ])

    #for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    
    #pop last layer and add DenseLayer of 2
    #model.pop()
    #model.add(Dense(2, activation='softmax'))
    return model

In [75]:
fc_bn_model = get_fc_bn_model()

make conv model layers non trainable and add fc_bn model to it

In [76]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_bn_model)

In [77]:
conv_model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
23000/23000 [==============================] - 651s - loss: 8.0620 - acc: 0.4994 - val_loss: 7.8898 - val_acc: 0.5105
Epoch 2/2
  300/23000 [..............................] - ETA: 593s - loss: 7.5755 - acc: 0.5300

KeyboardInterrupt: 

Since the accuracy I received for the fully connected batch normalized model was very less, trying the normal vgg model with last layer of Dense(2)

In [11]:
model.pop()
for layer in model.layers: layer.trainable=False
model.add(Dense(2, activation='softmax'))
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/4
23000/23000 [==============================] - 652s - loss: 0.1447 - acc: 0.9593 - val_loss: 0.0383 - val_acc: 0.9870
Epoch 2/4
23000/23000 [==============================] - 656s - loss: 0.1162 - acc: 0.9690 - val_loss: 0.0434 - val_acc: 0.9850
Epoch 3/4
23000/23000 [==============================] - 657s - loss: 0.1178 - acc: 0.9699 - val_loss: 0.0538 - val_acc: 0.9870
Epoch 4/4
23000/23000 [==============================] - 656s - loss: 0.1161 - acc: 0.9700 - val_loss: 0.0504 - val_acc: 0.9830


vgg model's val accuracy is .983 where as fc_bn_model had 0.51

Save this model

In [16]:
model.save_weights(base_path+"models/ft1.h5")

Now retrain all the Dense layers

In [21]:
layers = model.layers
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

conv_layers = layers[:last_conv_idx+1]
for layer in layers: layer.trainable = True
for layer in conv_layers: layer.trainable = False

In [22]:
for layer in model.layers:
    print layer.name,layer.trainable

lambda_1 False
zeropadding2d_1 False
convolution2d_1 False
zeropadding2d_2 False
convolution2d_2 False
maxpooling2d_1 False
zeropadding2d_3 False
convolution2d_3 False
zeropadding2d_4 False
convolution2d_4 False
maxpooling2d_2 False
zeropadding2d_5 False
convolution2d_5 False
zeropadding2d_6 False
convolution2d_6 False
zeropadding2d_7 False
convolution2d_7 False
maxpooling2d_3 False
zeropadding2d_8 False
convolution2d_8 False
zeropadding2d_9 False
convolution2d_9 False
zeropadding2d_10 False
convolution2d_10 False
maxpooling2d_4 False
zeropadding2d_11 False
convolution2d_11 False
zeropadding2d_12 False
convolution2d_12 False
zeropadding2d_13 False
convolution2d_13 False
maxpooling2d_5 True
flatten_1 True
dense_1 True
dropout_1 True
dense_2 True
dropout_2 True
dense_4 True


In [23]:
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/4
23000/23000 [==============================] - 653s - loss: 0.1123 - acc: 0.9713 - val_loss: 0.0538 - val_acc: 0.9855
Epoch 2/4
23000/23000 [==============================] - 656s - loss: 0.1060 - acc: 0.9740 - val_loss: 0.0506 - val_acc: 0.9855
Epoch 3/4
23000/23000 [==============================] - 657s - loss: 0.1213 - acc: 0.9718 - val_loss: 0.0635 - val_acc: 0.9820
Epoch 4/4
23000/23000 [==============================] - 657s - loss: 0.1219 - acc: 0.9713 - val_loss: 0.0563 - val_acc: 0.9845


In [24]:
model.save_weights(base_path+"models/ft2.h5")

In [25]:
base_path+"models/ft2.h5"

'/home/ubuntu/data/redux/models/ft2.h5'

In [28]:
def ConvBlock(model, layers, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))


def FCBlock(model, output_dim, activation, dropout):
    model.add(Dense(output_dim, activation=activation))
    model.add(Dropout(dropout))
        
def get_fc_bn_model2():
    FILE_PATH = '/home/ubuntu/data/redux/models/'

    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

    ConvBlock(model, 2, 64)
    ConvBlock(model, 2, 128)
    ConvBlock(model, 3, 256)
    ConvBlock(model, 3, 512)
    ConvBlock(model, 3, 512)
    
    model.add(Flatten())
    FCBlock(model, 4096, 'relu', 0.5)
    FCBlock(model, 4096, 'relu', 0.5)
    model.add(Dense(2, activation='softmax'))
        
    fname = 'ft2.h5'
    model.load_weights(FILE_PATH+fname)

    #pop the lastlayer
    #model.pop()

    #make all the layers non trainable
    #for layer in model.layers: layer.trainable = False
    #model.add(Dense(2, activation='softmax'))
    
    
    #pop the last 5 layers
    for i in range(0,5):
        model.pop()

    #make all the layers non trainable
    for layer in model.layers: layer.trainable = False
        
    FCBlock(model, 4096, 'relu', 0.5)
    FCBlock(model, 4096, 'relu', 0.5)  
    model.add(Dense(2, activation='softmax'))
    
    return model

In [29]:
fc_bn_model2 = get_fc_bn_model2()

In [30]:
fc_bn_model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 3, 224, 224)   0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
zeropadding2d_28 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_27[0][0]           
___________________________________________________________________________________________

In [31]:
fc_bn_model2.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fc_bn_model2.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=2, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
13500/23000 [================>.............] - ETA: 257s - loss: 8.0514 - acc: 0.4997

In [91]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
                                                                   lambda_1[0][0]          

#### Save the model

In [ ]:
model.save_weights(base_path+'model/final_model.h5')

### 7. Plot results

In [ ]:
cm = confusion_matrix(val_classes, preds)
plot_confusion_matrix(cm, {'cat':0, 'dog':1})